In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV 

import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
data = np.load('../data/train_50000.npz', allow_pickle = True)

In [3]:
from scipy import sparse

In [4]:
X = data['X'].item()
y = data['y']

In [5]:
X.shape

(500000, 1396)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [ ]:
# dtrain = xgb.DMatrix(X_train, y_train, feature_names=list([str(i) for i in range(X.shape[1])]))
# dtest = xgb.DMatrix(X_test, y_test, feature_names=list([str(i) for i in range(X.shape[1])]))

In [ ]:
import gc
del data
gc.collect()

In [ ]:
params1 = {'objective':['reg:linear'],
            'learning_rate': [.07], #so called `eta` value
            'n_estimators': [500],
            'max_depth': [5],
            'gamma': [0],
            'min_child_weight': [3],
            'subsample': [0.7],
            'colsample_bytree': [0.6],
            'reg_alpha':[1],
            'tree_method': ['gpu_hist'],
            'gpu_id': [0]}

In [ ]:
xgr = XGBRegressor(objective ='reg:linear',
            learning_rate = .07, #so called `eta` value
            n_estimators = 500,
            max_depth = 5,
            gamma = 0,
            min_child_weight = 3,
            subsample = 0.7,
            colsample_bytree = 0.6,
            reg_alpha = 1,
            tree_method = 'gpu_hist',
            gpu_id = 0)

In [ ]:
xgr.fit(X, y)

In [ ]:
import sys
sys.path.append('../tools/')
from model_manager import model_manager as mm

In [ ]:
xgb = mm('xgr', '../models/', xgr)

In [ ]:
xgb.load()

In [ ]:
res = xgb.model.apply(X)

In [ ]:
lr = LinearRegression(fit_intercept = True, normalize = False, copy_X=True, n_jobs = 5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(res, y, test_size=0.2, random_state=3)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
lr.fit(res, y)

In [ ]:
lr_mm = mm('Linear_regression', '../models/', lr)

In [ ]:
lr_mm.save()

In [ ]:
xgr = XGBRegressor()

xgb_grid = GridSearchCV(xgr, params1, cv = 3, n_jobs = 3, verbose=10)

In [ ]:
# gpu test
xgr = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
xgr.fit(X_train, y_train)

In [ ]:
xgb_grid.fit(X_train, y_train)

In [ ]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)